In [1]:
from common.functions import *
import numpy as np

### Q1. 손실함수 구현하기

In [2]:
# 3) 이제 본격적으로 손실함수를 구현하려고 합니다. 먼저, 오차제곱합을 구현하겠습니다. 수식은 아래와 같습니다. 아래 수식을 참고하여 오차 제곱합을 계산하는 것을 구현하세요. 함수 원형은 아래와 같습니다.(y, t는 각각 예측값과 실제값)
def sum_squares_error(y, t): #정답레이블뿐만 아니라, 오답레이블의 수치도 반영(원핫인코딩)
    return (0.5) * np.sum((y-t)**2)

In [3]:
# 4) 추가로, 교차 엔트로피(cross entropy)를 구현하려고 합니다. 아래 수식을 참고하여 교차 엔트로피를 계산하는 것을 구현하세요. 함수 원형은 아래와 같습니다. (단, tk와 yk는 각각 k번째 데이터의 실제 값과 예측 값)
#정답레이블이 원핫인코딩인 경우
def cross_entropy_error_one_hot(y, t):
    if y.dim==1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    batch_size = y.shape[0]
    delta = 1e-7
    return -np.sum(t*np.log(y+delta)) / batch_size #정답라벨링 이외의 값은 버려짐(*0)


#정답레이블이 단일값인 경우
def cross_entropy_error(y, t): #이 구현의 경우 원핫인코딩에서 t가 0인 원소는 교차엔트로피 오차도 0이므로 그 계산을 무시한다는 것이 핵심. 즉 정답에 해당하는 신경망의 출력만으로 교차 엔트로피 오차를 계산!
    if y.dim==1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    batch_size = y.shape[0]
    delta = 1e-7
    return -np.sum(np.log(y[np.arrange(batch_size), t] + delta)) / batch_size 

### Q2. 수치 미분 구현하기

In [4]:
# 2) 이제 편미분을 구현하려고 합니다. 목적함수 f와 미지수 x가 주어졌을 때, 편미분을 하는 과정을 구현하세요. 함수 원형은 아래와 같습니다. (단, 구현하실 때 가중치가 1차원의 형태가 아닌 2차원의 형태가 들어왔다고 가정하고 구현하셔야 합니다.) hint : np.nditer 활용
def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 값 복원
        it.iternext()   
        
    return grad

In [5]:
# 3) 매개변수를 갱신하기 위해 경사 하강법(gradient descent)를 이용하려고 합니다. 경사하강법을 적용할 때 갱신되는 매개변수의 식은 아래와 같습니다. 𝒙𝒌+𝟏 = 𝒙𝒌 − 𝜶 × 𝝏𝒇/𝝏𝒙𝒌 위 식을 이용하여 경사 하강법을 구현하세요. 함수 원형은 아래와 같습니다. (f : 목적함수, init_x : 매개변수의 초기값, lr : alpha(학습률), step_num : 경사하강법을 진행할 횟수)
def gradient_descent(f, init_x, lr=0.01, step_num=100):
    x = init_x
    for i in range(step_num):
        grad = numerical_gradient(f,x) #"이만큼 이동하세요"
        x -= lr*grad
    return x

### Q3. 2층 신경망을 통해 가중치 업데이트하기
MNIST 데이터를 이용하여 아래와 같은 신경망을 구현해 보려고 합니다.(순서대로 아래 쪽 숫자는 784, 100, 10입니다.)

In [6]:
# 1) 아래와 같은 코드를 불러오세요.
import sys, os
sys.path.append(os.pardir)
from common.functions import *
from common.gradient import numerical_gradient

In [8]:
# 2) 이제 위 그림과 같은 2층 신경망을 구현하여 MNIST 데이터에 대하여 가중치를 업데이트 해보려고 합니다. 이 과정은 TwoLayerNet이라는 클래스 안에서 진행될 예정입니다. Ipynb 파일 안에 TwoLayerNet이라는 이름의 클래스를 하나 만들어 주세요

In [8]:
# 3) 2번에서 만든 TwoLayerNet 클래스 안에 아래와 같은 함수를 구현하려고 합니다. 각 함수의 설명을 참고하여 알맞게 코드를 구현해 주세요.(weight_init_std도 활용해주세요)
import sys, os
sys.path.append(os.pardir)
from common.functions import *
from common.gradient import numerical_gradient

class TwoLayerNet:
    # a. 해당 함수는 가중치를 초기화 하는 함수입니다. Input_size는 입력층의 노드, hidden_size는 은닉층의 노드, output_size는 출력층의 노드 수 입니다. 이를 참고한 후 딕셔너리를 이용하여 가중치를 초기화하는 함수를 구현하세요. (해당 함수는 클래스에서 초기화가 일어나는 부분이기 때문에 리턴값이 없습니다.)
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size,hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size,output_size)
        self.params['b2'] = np.zeros(output_size)
    
    # b. 해당 함수는 초기화된 가중치를 이용하여 신경망의 연산 과정이 일어나는 부분입니다. 지난 주와 동일한 방법으로 결과를 리턴하도록 구현하세요. 리턴해야 하는 것은 y(예측값) 입니다.
    def predict(self, x):
        W1,W2 = self.params['W1'], self.params['W2']
        b1,b2 = self.params['b1'], self.params['b2']
        a1 = np.dot(x,W1)+b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1,W2)+b2
        y = softmax(a2)
        return y
    
    # c. 해당 함수는 예측 결과와 실제 정답 간의 정확도를 계산하는 부분입니다. X와 t는 각각 입력 데이터와 정답 레이블입니다. 이를 참고하여 정확도 accuracy를 계산하고 리턴하세요.
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y,axis=1)
        t = np.argmax(t,axis=1)
        accuracy = np.sum(y==t) / float(x.shape[0])
        return accuracy

    # d. 해당 함수는 예측 결과와 실제 정답 간의 손실을 계산하는 부분입니다. 계산 과정을 구현하고 loss를 리턴하세요. Loss function은 자율적으로 선택하시고, 해당 loss function을 선택한 이유를 설명하세요.
    def loss(self, x, t):
        y = self.predict(x)
        return cross_entropy_error(y,t)

    # e. 해당 함수는 계산한 loss를 바탕으로 가중치를 업데이트하는 함수입니다. 딕셔너리를 이용하여 업데이트된 가중치를 리턴하도록 구현하세요.
    def numerical_gradient(self, x, t):
        loss_W = lambda W:self.loss(x,t)
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads

In [9]:
# 4) 이제 3번에서 구현한 클래스를 바탕으로 연산 과정을 해보려고 합니다. 아래 코드를 실행하여서 가중치의 shape가 올바르게 출력되는 지 확인하세요.
#from common.two_layer_net import * # 위 함수와 똑같지만 충돌발생하기때문에 주석처리
net = TwoLayerNet(input_size = 784, hidden_size = 100, output_size = 10)
print(net.params['W1'].shape, net.params['b1'].shape, net.params['W2'].shape, net.params['b2'].shape)

(784, 100) (100,) (100, 10) (10,)


In [21]:
# 6) 본격적으로 MNIST 데이터를 이용하여 가중치를 업데이트 해보려고 합니다.(train data 이용) Epochs는 2번만 돌고, 한 epoch당 처리하는 배치의 크기(batch size)는 100으로 지정하여 가중치가 업데이트 되는 과정을 구현합니다. 또한, 가중치 w1이 초기값과 얼마나 차이가 나는 지를 출력하는 코드를 작성하세요.
import numpy as np
import sys
sys.path.append('')
from dataset.mnist import load_mnist
#from common.two_layer_net import TwoLayerNet

import copy

(x_train, t_train), (x_test, t_test) = load_mnist(normalize = True, one_hot_label = True)

#일부만 가져오기
subset_size = 10000
x_train = x_train[:subset_size]
t_train = t_train[:subset_size]


train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []
iter_per_epoch = max(train_size / batch_size, 1)


epoch_num = 2  # 원하는 epoch 수
iter_num = int(epoch_num * train_size / batch_size)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)
init_W1 = copy.deepcopy(network.params['W1']) #비교를 위한 초기값 저장


for i in range(iter_num):
    batch_mask = np.random.choice(train_size,batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    grad = network.numerical_gradient(x_batch, t_batch)

    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate*grad[key]

    loss = network.loss(x_batch,t_batch)
    train_loss_list.append(loss)

    if i == 0 or (i+1) % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train,t_train)
        test_acc = network.accuracy(x_test,t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc : " + str(train_acc) + ", " + str(test_acc))

train acc, test acc : 0.0944, 0.0974
train acc, test acc : 0.4202, 0.4011
train acc, test acc : 0.6146, 0.6053


In [22]:
# W1 초기값 (비교를 위해 sum연산된 값으로 출력)
print(np.sum(init_W1))

# W1 현재값
print(np.sum(network.params['W1']))

# 차이 비교
print(np.sum(init_W1 - network.params['W1']))

-3.8557096720821074
-61.76200638274949
57.90629671066738


In [23]:
train_loss_list

[8.789917477035026,
 9.402878615095732,
 8.53129252003895,
 8.956632884256209,
 9.155539960960349,
 9.17339682078703,
 9.393068895443568,
 9.32320104269466,
 8.871058476502549,
 8.748610338629673,
 9.860914834073586,
 8.708943996540238,
 8.528062875459552,
 9.190757737328298,
 8.668473667912115,
 8.74836678599071,
 9.628166929402846,
 9.014904746447389,
 8.920513072717156,
 8.636347240924136,
 8.494875067946344,
 8.504336803104428,
 9.415620445784853,
 8.532462989336825,
 8.251651851354955,
 8.334178857515965,
 7.977338374885249,
 8.508595628327468,
 9.037225948140746,
 8.52239796478214,
 9.222595521753204,
 9.103690415217098,
 8.989468279386891,
 7.676927544145608,
 8.539393538062177,
 8.981825598728424,
 7.954785772585041,
 7.660103009255141,
 8.763949356144254,
 8.86246195821568,
 7.975586171879179,
 7.947219773391989,
 7.339098352617026,
 9.056825281589106,
 7.326638564813185,
 7.176040996159823,
 8.178553661202754,
 7.7584252656771175,
 7.359936547510157,
 7.742716069808982,
 7.45

In [24]:
# 7) 추가로, 하나의 에폭마다 학습 정확도와 테스트 정확도를 저장할 수 있도록 코드를 추가하세요.(교재 p.144 참고)
print(train_acc_list)
print(test_acc_list)

[0.0944, 0.4202, 0.6146]
[0.0974, 0.4011, 0.6053]


### Q4. 3층 신경망 구현하기
이번엔 3층 신경망의 가중치를 업데이트 하는 과정을 해보려고 합니다. Q3과 동일한 과정을 거치면 됩니다. 단, 이번 신경망의 구조는 아래와 같습니다.(input layer의 노드는 784이고,은닉층은 각각 100, 50이며 마지막 출력층 노드는 10입니다.)


In [10]:
# 1) 이제 위 그림과 같은 3층 신경망을 구현하여 MNIST 데이터에 대하여 가중치를 업데이트 해보려고 합니다. 이 과정은 ThreeLayerNet이라는 클래스 안에서 진행될 예정입니다. Ipynb 파일 안에 ThreeLayerNet이라는 이름의 클래스를 하나 만들어 주세요
class ThreeLayerNet():
# 2) 1번에서 만든 TwoLayerNet 클래스 안에 아래와 같은 함수를 구현하려고 합니다. 각 함수의 설명을 참고하여 알맞게 코드를 구현해 주세요.
    # a. 해당 함수는 가중치를 초기화 하는 함수입니다. Input_size는 입력층의 노드, hidden_size는 은닉층의 노드, output_size는 출력층의 노드 수 입니다. 은닉층의 노드 수는 list의 형태로 저장됩니다. (ex.[100, 50]) 이를 참고한 후 딕셔너리를 이용하여 가중치를 초기화 하는 함수를 구현하세요. (해당 함수는 클래스에서 초기화가 일어나는 부분이기 때문에 리턴값이 없습니다.)
    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size,hidden_size[0])
        self.params['b1'] = np.zeros(hidden_size[0])
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size[0],hidden_size[1])
        self.params['b2'] = np.zeros(hidden_size[1])
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size[1],output_size)
        self.params['b3'] = np.zeros(output_size)

    # b. Q3처럼 predict, accuracy, loss, numerical_gradient 함수를 채워주세요.
    def predict(self, x):
        W1,W2,W3 = self.params['W1'], self.params['W2'], self.params['W3']
        b1,b2,b3 = self.params['b1'], self.params['b2'], self.params['b3']
        a1 = np.dot(x,W1)+b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1,W2)+b2
        z2 = sigmoid(a2)
        a3 = np.dot(z2,W3)+b3
        y = softmax(a3)
        return y

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y,axis=1)
        t = np.argmax(t,axis=1)
        accuracy = np.sum(y==t) / float(x.shape[0])
        return accuracy

    def loss(self, x, t):
        y = self.predict(x)
        return cross_entropy_error(y,t)

    def numerical_gradient(self, x, t):
        loss_W = lambda W:self.loss(x,t)
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        grads['W3'] = numerical_gradient(loss_W, self.params['W3'])
        grads['b3'] = numerical_gradient(loss_W, self.params['b3'])
        return grads

In [11]:
# 3) 지금까지 구현한 ThreeLayerNet을 실행시키려고 합니다. 아래 코드를 실행하여 가중치의 shape가 올바르게 출력되는 지 확인하세요.
net = ThreeLayerNet(input_size = 784, hidden_size = [100, 50], output_size = 10)
print(net.params['W1'].shape, net.params['b1'].shape, net.params['W2'].shape, net.params['b2'].shape, net.params['W3'].shape, net.params['b3'].shape)

(784, 100) (100,) (100, 50) (50,) (50, 10) (10,)


In [37]:
# 4) 본격적으로 MNIST 데이터를 이용하여 가중치를 업데이트 해보려고 합니다. Epochs는 1번만 돌고, 한 epoch당 처리하는 배치의 크기(batch size)는 128으로 지정하여 가중치가 업데이트 되는 과정을 구현하세요. 해당 코드 아래쪽에 이어서 구현하시면 됩니다.
import numpy as np
from dataset.mnist import load_mnist
(x_train, t_train), (x_test, t_test) = load_mnist(normalize = True, one_hot_label = True)
import copy

#일부만 가져오기
subset_size = 12800
x_train = x_train[:subset_size]
t_train = t_train[:subset_size]


train_size = x_train.shape[0]
batch_size = 128
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []
iter_per_epoch = max(train_size / batch_size, 1)

epoch_num = 1  # 원하는 epoch 수
iter_num = int(epoch_num * train_size / batch_size)
network = ThreeLayerNet(input_size = 784, hidden_size = [100, 50], output_size = 10)
init_W1 = copy.deepcopy(network.params['W1']) #비교를 위한 초기값 저장


for i in range(iter_num):
    batch_mask = np.random.choice(train_size,batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    grad = network.numerical_gradient(x_batch, t_batch)

    for key in ('W1', 'b1', 'W2', 'b2', 'W3', 'b3'):
        network.params[key] -= learning_rate*grad[key]

    loss = network.loss(x_batch,t_batch)
    train_loss_list.append(loss)

    if i == 0 or (i+1) % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train,t_train)
        test_acc = network.accuracy(x_test,t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc : " + str(train_acc) + ", " + str(test_acc))

train acc, test acc : 0.1, 0.0958
train acc, test acc : 0.099375, 0.1009


In [38]:
# W1 초기값 (비교를 위해 sum연산된 값으로 출력)
print(np.sum(init_W1))

# W1 현재값
print(np.sum(network.params['W1']))

# 차이 비교
print(np.sum(init_W1 - network.params['W1']))

2.899649556691728
4.8433396174813765
-1.943690060789649


In [40]:
train_loss_list

[9.22846154506918,
 9.409232733207835,
 9.847872665308344,
 8.978458827239526,
 8.997127235349055,
 8.801956561860145,
 9.095912950727836,
 9.171282628885649,
 9.52650694477084,
 9.734651494213777,
 8.937961408308698,
 9.382178287653707,
 9.395780958308205,
 9.031771217557571,
 9.102811321227929,
 8.0044743879103,
 9.13097660340922,
 9.64298405058637,
 9.237727476739007,
 8.04459836522955,
 9.097296447624327,
 9.449206220933014,
 8.985048929437028,
 10.052724146642756,
 9.22948947336451,
 9.346240115553359,
 9.000379750914746,
 8.718195814509336,
 9.090859973674046,
 8.14259067455781,
 9.247117402059562,
 8.691616095679962,
 8.848032347069427,
 8.226556199125277,
 9.21362526100964,
 8.744284569970826,
 9.058842931015644,
 8.3283691415599,
 8.93792631471601,
 8.31640314223811,
 9.603657928657151,
 9.27610002391576,
 9.084423651075804,
 8.044630383502847,
 8.819819970824728,
 8.534620527974822,
 8.72364827926604,
 9.479959925130075,
 9.053444732350824,
 8.995175697074982,
 9.290865552906

In [39]:
print(train_acc_list)
print(test_acc_list)

[0.1, 0.099375]
[0.0958, 0.1009]


# (3층신경망) 활성화 함수만 sigmoid에서 relu로 바꾸고 재실행!!

In [7]:
class ThreeLayerNet():
    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size,hidden_size[0])
        self.params['b1'] = np.zeros(hidden_size[0])
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size[0],hidden_size[1])
        self.params['b2'] = np.zeros(hidden_size[1])
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size[1],output_size)
        self.params['b3'] = np.zeros(output_size)

    # b. Q3처럼 predict, accuracy, loss, numerical_gradient 함수를 채워주세요.
    def predict(self, x):
        W1,W2,W3 = self.params['W1'], self.params['W2'], self.params['W3']
        b1,b2,b3 = self.params['b1'], self.params['b2'], self.params['b3']
        a1 = np.dot(x,W1)+b1
        z1 = relu(a1)
        a2 = np.dot(z1,W2)+b2
        z2 = relu(a2)
        a3 = np.dot(z2,W3)+b3
        y = softmax(a3)
        return y

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y,axis=1)
        t = np.argmax(t,axis=1)
        accuracy = np.sum(y==t) / float(x.shape[0])
        return accuracy

    def loss(self, x, t):
        y = self.predict(x)
        return cross_entropy_error(y,t)

    def numerical_gradient(self, x, t):
        loss_W = lambda W:self.loss(x,t)
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        grads['W3'] = numerical_gradient(loss_W, self.params['W3'])
        grads['b3'] = numerical_gradient(loss_W, self.params['b3'])
        return grads
        



import numpy as np
from dataset.mnist import load_mnist
(x_train, t_train), (x_test, t_test) = load_mnist(normalize = True, one_hot_label = True)
import copy

#일부만 가져오기
subset_size = 12800
x_train = x_train[:subset_size]
t_train = t_train[:subset_size]


train_size = x_train.shape[0]
batch_size = 128
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []
iter_per_epoch = max(train_size / batch_size, 1)

epoch_num = 1  # 원하는 epoch 수
iter_num = int(epoch_num * train_size / batch_size)
network = ThreeLayerNet(input_size = 784, hidden_size = [100, 50], output_size = 10)
init_W1 = copy.deepcopy(network.params['W1']) #비교를 위한 초기값 저장


for i in range(iter_num):
    batch_mask = np.random.choice(train_size,batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    grad = network.numerical_gradient(x_batch, t_batch)

    for key in ('W1', 'b1', 'W2', 'b2', 'W3', 'b3'):
        network.params[key] -= learning_rate*grad[key]

    loss = network.loss(x_batch,t_batch)
    train_loss_list.append(loss)

    if i == 0 or (i+1) % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train,t_train)
        test_acc = network.accuracy(x_test,t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc : " + str(train_acc) + ", " + str(test_acc))

train acc, test acc : 0.099375, 0.1009
train acc, test acc : 0.3690625, 0.3634


In [8]:
# W1 초기값 (비교를 위해 sum연산된 값으로 출력)
print(np.sum(init_W1))

# W1 현재값
print(np.sum(network.params['W1']))

# 차이 비교
print(np.sum(init_W1 - network.params['W1']))

-1.9509526464380271
-28.780734249493413
26.82978160305538


In [9]:
train_loss_list

[9.397513826800129,
 11.31209118987985,
 9.933224544520629,
 9.287576777032472,
 9.893111019232025,
 8.842578211236898,
 10.263969898714013,
 9.679057973628723,
 9.724913153112322,
 9.053697796163702,
 10.07288389720543,
 9.721100220761677,
 9.61998597695606,
 9.149386527674409,
 9.098549353965385,
 9.852076822098592,
 9.248633961284355,
 9.187945329580826,
 9.074728162624783,
 9.628564686103992,
 10.118749197917882,
 9.149461089323037,
 10.000946806610678,
 9.01518834076326,
 10.133714707199642,
 8.981618896214295,
 9.443771134050797,
 9.160638610309165,
 9.486961193397693,
 9.606360774425275,
 9.395923528506437,
 9.465657072767298,
 9.036176668261703,
 9.716513639817425,
 9.569464892056445,
 8.977453311460472,
 8.752065279162187,
 9.118176660016612,
 8.817604205635774,
 9.336090747622801,
 8.269176183613673,
 7.868050534902874,
 9.197797757070745,
 9.359002361181794,
 8.923744571311353,
 9.26156107512939,
 9.134836027178835,
 8.943283866597366,
 9.184607053355712,
 9.267559903420771,

In [11]:
print(train_acc_list)
print(test_acc_list)

[0.099375, 0.3690625]
[0.1009, 0.3634]


#### 3층신경망의 경우 데이터가 적어서인지(시간상 일부만사용), 학습횟수가 더 많이 필요해서인지 모르겠지만, 2층신경망과 다르게 1epoch만에 유의한 성능개선이 이루어지지않음.
#### 다만, 활성화함수를 sigmoid에서 relu로 변경 후 실행해보니 성능향상을 확인할 수 있었음. (층이 깊어질수록 발생하는 기울기소실문제가 있었을 가능성)